In [1]:
from __future__ import print_function

import tensorflow as tf
import tensornets as nets

from tensornets.datasets import coco

data_dir = '/home/taehoonlee/Data/COCO'
data_name = 'val2014'

In [2]:
inputs1 = tf.placeholder(tf.float32, [None, 416, 416, 3])
inputs2 = tf.placeholder(tf.float32, [None, 608, 608, 3])
models = []

with tf.device('gpu:0'):
    models += [nets.YOLOv2COCO(inputs1)]
    models += [nets.YOLOv2COCO(inputs2)]

with tf.device('gpu:1'):
    models += [nets.YOLOv3COCO(inputs1)]
    models += [nets.YOLOv3COCO(inputs2)]

In [3]:
imgs = coco.load(data_dir, data_name)
results = [[] for _ in range(len(models))]

with tf.Session() as sess:
    for m in models:
        sess.run(m.pretrained())
    for (k, (img, scale)) in enumerate(imgs):
        outs = sess.run(models[:-2] + models[-2].preds + models[-1].preds,
                        {inputs1: models[0].preprocess(img),
                         inputs2: models[1].preprocess(img)})
        outs = outs[:-6] + [outs[-6:-3]] + [outs[-3:]]
        for i in range(len(models)):
            results[i].append(models[i].get_boxes(outs[i], img.shape[1:3]))
        if k % 10 == 0:
            print("%d" % k, end='\r')

loading annotations into memory...
Done (t=6.49s)
creating index...
index created!


In [4]:
for result in results:
    print(coco.evaluate(result, data_dir, data_name))

| person | bicycl |    car | motorb | aeropl |    bus |  train |  truck |   boat | traffi | fire h | stop s | parkin |  bench |   bird |    cat |    dog |  horse |  sheep |    cow | elepha |   bear |  zebra | giraff | backpa | umbrel | handba |    tie | suitca | frisbe |   skis | snowbo | sports |   kite | baseba | baseba | skateb | surfbo | tennis | bottle | wine g |    cup |   fork |  knife |  spoon |   bowl | banana |  apple | sandwi | orange | brocco | carrot | hot do |  pizza |  donut |   cake |  chair |   sofa | potted |    bed | dining | toilet | tvmoni | laptop |  mouse | remote | keyboa | cell p | microw |   oven | toaste |   sink | refrig |   book |  clock |   vase | scisso | teddy  | hair d | toothb |
|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|-------